In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)


In [2]:
import tensorflow as tf 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random
from keras.preprocessing.image import ImageDataGenerator

In [4]:
TRAIN_PATH = r'E:\Leuk study re-designed\ALLIDB-2\Low imbalance\Train - 1 to 10 ratio\all'

BATCH_SIZE=1

In [ ]:
import skimage
from skimage.io import imread, imshow
data = []
i = 0
CATEGORIES = r'all'
plt.figure(figsize=(15, 15))
#for category in CATEGORIES:
#path = os.path.join(TRAIN_PATH, category)
#print(path)
list_img = os.listdir(TRAIN_PATH)
#print(list_img)
for img in list_img:
    img_path = os.path.join(TRAIN_PATH, img)
    print(img_path)

    #label = CATEGORIES.index(category)
    arr = imread(img_path)
    #imshow(arr)
    #break
    #crop_arr = crop_center(arr, (210,210))
    #if 1 <= i+1 <= 140:                      # total 140 image
    #    ax = plt.subplot(13, 11, i+1)
    #plt.imshow(arr)
    #i += 1
    data.append(arr)

In [ ]:
len(data)

In [ ]:
random.shuffle(data)

x_train = []
#y_train = []

for features in data:
    x_train.append(features)
    #y_train.append(label)

    
x_train = np.array(x_train)
#y_train = np.array(y_train)
print(x_train.shape)
print(x_train.dtype)

r=c=452
import skimage
from skimage.transform import resize
new_x_train = np.zeros((x_train.shape[0], r, c, 3), dtype=np.uint8)
print(new_x_train.shape)
i=0
for img in x_train:
    image = resize(img,(r, c, 3), preserve_range=True)
    new_x_train[i] = image
    i+=1
    #print(i)

new_x_train = new_x_train/255.0
new_x_train = new_x_train.astype('float32')
print(new_x_train.shape)
print(new_x_train.dtype)


#np.save("x_train_leuknet",x_train)
#np.save("y_train_leuknet",y_train)

In [ ]:
import skimage
from skimage.io import imread, imshow
imshow(new_x_train[0])

In [9]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Permute, multiply
import tensorflow.keras.backend as K


def squeeze_excite_block(tensor, ratio=16):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

# Original net

In [ ]:
'''
#Original net
import keras
from keras import layers

input_img = keras.Input(shape=(452, 452, 3))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = squeeze_excite_block(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='Adagrad', loss='binary_crossentropy') #metrics=['accuracy']
autoencoder.summary()
'''

# Try 1

In [ ]:
#'''
#Original net
import keras
from keras import layers

input_img = keras.Input(shape=(452, 452, 3))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer="he_normal")(input_img)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer="he_normal")(x)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer="he_normal")(x)
x = squeeze_excite_block(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer="he_normal")(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer="he_normal")(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal")(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same', kernel_initializer="he_normal")(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='Adam', loss='binary_crossentropy') #metrics=['accuracy']
autoencoder.summary()
#'''

In [ ]:
'''
#Modified
import keras
from keras import layers

input_img = keras.Input(shape=(252, 252, 3))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
#x = layers.Conv2D(16, (3, 3), activation='relu')(x)
#x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
autoencoder.summary()

'''

In [ ]:
'''
#Big-one
import keras
from keras import layers

input_img = keras.Input(shape=(r, c, 3))

x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (2, 2, 8) i.e. 32-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
#x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()
'''

In [ ]:
'''
import tensorflow as tf
#from t.keras.optimizers import Adam, RMSprop, SGD
adam_opt = tf.keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-4)
#sgd_opt = tf.keras.optimizers.SGD(lr=1e-06, momentum=0.9, decay=0.0, nesterov=False)
#rmsp_opt = tf.keras.optimizers.RMSprop(lr=1e-4, decay=0.9)
# eve_opt = Eve(lr=1e-4, decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08)

autoencoder.compile(optimizer= adam_opt,
              loss = 'binary_crossentropy',
              metrics=['accuracy'])
'''

In [15]:
#import keras
import tensorflow.keras as keras
#from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('train_norm_autoencoder_200.h5', monitor='loss', save_best_only=True, mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.0, verbose=1, patience=5, mode='max')]


In [16]:
#from keras.preprocessing.image import ImageDataGenerator
#dataAugmentaion = ImageDataGenerator()

In [ ]:
'''
autoencoder.fit(dataAugmentaion.flow(new_x_train, new_x_train, batch_size = 2),
                epochs=200,
                #batch_size=128,
                shuffle=True,
                callbacks = callbacks)
'''


In [ ]:
#'''

autoencoder.fit(new_x_train, new_x_train,
                epochs=500,
                batch_size=4,
                shuffle=True,
                callbacks = callbacks)
                #validation_data=(x_test, x_test))
                #callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])
#'''

In [ ]:
'''
import tensorflow as tf
from skimage.io import imsave, imshow
from skimage.transform import resize
model = tf.keras.models.load_model('./train_norm_autoencoder_200.h5')
#model = autoencoder

img_names = os.listdir(r'D:\new_leuk\Leukemia_Work_Revive\dataset\val\all')
print(type(model))

test_preds = np.zeros((new_x_train.shape[0], new_x_train.shape[1], new_x_train.shape[2], new_x_train.shape[3]), dtype=np.float32)
'''
'''
for x in range(new_x_train.shape[0]):
    x_test = new_x_train[x]
    x_test = np.expand_dims(x_test, axis=0)

    test_preds[x] = autoencoder.predict(x_test)
    print(type(test_preds[x]))
    print(test_preds[x].shape)
    
    #imshow(test_preds[x])
    
    #break
    
    print(img_names[x][:-4])
    path = 'D:/new_leuk/Leukemia_Work_Revive/Autoencoder oversample data/val/all' + '/' + img_names[x][:-4] + '_syn.bmp'
    print(path)
    imsave(path, resize(test_preds[x].astype('float32'), (450, 450, 3), preserve_range=True))
    if x==4:
        break
''' 
   
    

'''
x_test = new_x_train[50]
imshow(new_x_train[50])
x_test = np.expand_dims(x_test, axis=0)
x_test.shape

nx_test = x_test[0,:,:,:]
nx_test.shape
imshow(nx_test)
'''

In [ ]:
autoencoder.save('./final_hem_autoencoder.h5')

In [ ]:
#set_test.shape

In [ ]:
''''''
import tensorflow as tf
from skimage.io import imsave, imshow
from skimage.transform import resize
model = tf.keras.models.load_model('./final_hem_autoencoder.h5')
#model = tf.keras.models.load_model('train_norm_autoencoder_200.h5')
#model = autoencoder

img_names = os.listdir(r'E:\Leuk study re-designed\ALLIDB-2\Low imbalance\Train - 1 to 10 ratio\all')
img_names.sort()
print(type(model))



PATH = r'E:\Leuk study re-designed\ALLIDB-2\Low imbalance\Train - 1 to 10 ratio\all'
import skimage 
from skimage.io import imread

set_test = np.zeros((338,452,452,3), dtype=np.uint8) #hem batch 648,  #all batch 1,219
for x in range(len(img_names)):
    img = imread(os.path.join(PATH, img_names[x]))
    img = resize(img,(new_x_train.shape[1], new_x_train.shape[2], new_x_train.shape[3]), preserve_range=True)
    set_test[x] = img

set_test = set_test/255.0
set_test = set_test.astype('float32')
print(set_test.shape)
    
    
    
test_preds = np.zeros((new_x_train.shape[0], new_x_train.shape[1], new_x_train.shape[2], new_x_train.shape[3]), dtype=np.float32)


for x in range(new_x_train.shape[0]):
    x_test = set_test[x]
    #x_test = new_x_train[x]
    x_test = np.expand_dims(x_test, axis=0)

    test_preds[x] = autoencoder.predict(x_test)
    #test_preds[x] = model.predict(x_test)
    print(type(test_preds[x]))
    print(test_preds[x].shape)
    
    #imshow(test_preds[x])
    
    #break
    #'''
    print(img_names[x][:-4])
    path = 'E:/Leuk study re-designed/ALLIDB-2/Low imbalance/CycleGan Oversample/all' + '/' + img_names[x][:-4] + '_syn.tif'
    print(path)
    imsave(path, resize(test_preds[x].astype('float32'), (450, 450, 3), preserve_range=True))
    #if x==4:
    #    break
    #''' 
   
    

'''
x_test = new_x_train[50]
imshow(new_x_train[50])
x_test = np.expand_dims(x_test, axis=0)
x_test.shape

nx_test = x_test[0,:,:,:]
nx_test.shape
imshow(nx_test)
'''